In [ ]:
#ground state
#clingo code

str1 = """
    %Instances
    a(1,1,8).
    a(2,3,3).
    a(2,4,6).
    a(3,2,7).
    a(3,5,9).
    a(3,7,2).
    a(4,2,5).
    a(4,6,7).
    a(5,5,4).
    a(5,6,5).
    a(5,7,7).
    a(6,4,1).
    a(6,8,3).
    a(7,3,1).
    a(7,8,6).
    a(7,9,8).
    a(8,3,8).
    a(8,4,5).
    a(8,8,1).
    a(9,2,9).
    a(9,7,4).
""" 

#Constraints
str2 = """
    % Generate
    % Each number 1..9 is assigned to one cell in each box
    1 { a(X,Y,N):X=1..9,Y=1..9,X1<=X,X<=X1+2,Y1<=Y,Y<=Y1+2 } 1
    :- N=1..9, X1 = 3*(0..2)+1, Y1 = 3*(0..2)+1.
    % no two different numbers given a row and a column
    :- a(X,Y,N), a(X,Y,N1), N!=N1.
    % no two different columns given a row and a number
    :- a(X,Y,N), a(X,Y1,N), Y!=Y1.
    % no two different rows given a column and a number
    :- a(X,Y,N), a(X1,Y,N), X!=X1.    
"""

# Concat both as asp_code_base
asp_code_base = f"{str1.strip()}{str2.strip()}"



In [ ]:
import clingo
import numpy as np
import re
from copy import deepcopy



# function to extract the cell values for each instance and add it to a 9x9 list
def UpdateGrid(strng, grid):
    temp = re.findall(r'\d+', strng)
    res = list(map(int, temp))
    res = np.array(res)
    lst = res.reshape((-1,3))
    for it in range(len(lst)):
        i = lst[it][0]-1
        j = lst[it][1]-1
        grid[i][j] = lst[it][2]
    return grid

# function to run the clingo code and update the grid with the sudoko solution
def on_model(model):
    #print("Found solution:", model)
    x = model.__str__()
    UpdateGrid(x, grid1)
        

In [ ]:
# creating a zeros 9x9 grid (list)
grid = np.zeros((9,9),dtype = int)
grid = grid.tolist()

# update the initial state to the 9x9 list
UpdateGrid(str1, grid)
# store the initial grid in grid0 variable to be able to restart the game
grid0 = deepcopy(grid)


control = clingo.Control()
control.configuration.solve.models = 1  # find one answer only

control.add("base", [], asp_code_base)

parts = []
parts.append(("base", []))
control.ground(parts)

ret = None

control.cleanup()  # cleanup previous grounding call, so we can ground again
control.ground(parts)

# for animation purposes, we will just store the final grid as grid1 and then use it to update 
# the values cell by cell when we run PyGame
grid1 = deepcopy(grid0)

ret = control.solve(on_model=on_model)

In [ ]:
# import pygame library
import pygame
 
# initialise the pygame font
pygame.font.init()
 
# Total window
screen = pygame.display.set_mode((500, 600))
 
# Title and Icon
pygame.display.set_caption("SUDOKU SOLVER USING ASP")
img = pygame.image.load('icon.png')
pygame.display.set_icon(img)
 
x = 0
y = 0
dif = 500 / 9
val = 0
 
# Load test fonts for future use
font1 = pygame.font.SysFont("comicsans", 28)
font2 = pygame.font.SysFont("comicsans", 10)
def get_cord(pos):
    global x
    x = pos[0]//dif
    global y
    y = pos[1]//dif
 
# Highlight the cell selected
def draw_box():
    for i in range(2):
        pygame.draw.line(screen, (255, 0, 0), (x * dif-3, (y + i)*dif), (x * dif + dif + 3, (y + i)*dif), 7)
        pygame.draw.line(screen, (255, 0, 0), ( (x + i)* dif, y * dif ), ((x + i) * dif, y * dif + dif), 7)  
 
# Function to draw required lines for making Sudoku grid        
def draw():
    # Draw the lines
    for i in range (9):
        for j in range (9):
            if grid[i][j]!= 0:
 
                # Fill blue color in already numbered grid
                pygame.draw.rect(screen, (0, 153, 153), (i * dif, j * dif, dif + 1, dif + 1))
 
                # Fill grid with default numbers specified
                text1 = font1.render(str(grid[i][j]), 1, (0, 0, 0))
                screen.blit(text1, (i * dif + 15, j * dif + 15))
    # Draw lines horizontally and verticallyto form grid          
    for i in range(10):
        if i % 3 == 0 :
            thick = 7
        else:
            thick = 1
        pygame.draw.line(screen, (0, 0, 0), (0, i * dif), (500, i * dif), thick)
        pygame.draw.line(screen, (0, 0, 0), (i * dif, 0), (i * dif, 500), thick)     
 
# Fill value entered in cell     
def draw_val(val):
    text1 = font1.render(str(val), 1, (0, 0, 0))
    screen.blit(text1, (x * dif + 15, y * dif + 15))   
 
# Raise error when wrong value entered
def raise_error1():
    text1 = font1.render("WRONG !!!", 1, (0, 0, 0))
    screen.blit(text1, (20, 570)) 
def raise_error2():
    text1 = font1.render("Wrong !!! Not a valid Key", 1, (0, 0, 0))
    screen.blit(text1, (20, 570)) 
 
# Check if the value entered in board is valid
def valid(m, i, j, val):
    for it in range(9):
        if m[i][it]== val:
            return False
        if m[it][j]== val:
            return False
    it = i//3
    jt = j//3
    for i in range(it * 3, it * 3 + 3):
        for j in range (jt * 3, jt * 3 + 3):
            if m[i][j]== val:
                return False
    return True

def solve(grid, i, j):
     
    while grid[i][j]!= 0:
        if i<8:
            i+= 1
        elif i == 8 and j<8:
            i = 0
            j+= 1
        elif i == 8 and j == 8:
            return True
    pygame.event.pump()   
    grid[i][j]= grid1[i][j]    
    global x, y
    x = i
    y = j
    # white color background\
    screen.fill((255, 255, 255))
    draw()
    draw_box()
    pygame.display.update()
    pygame.time.delay(20)
    if solve(grid, i, j)== 1:
        return True

    return False 

# Display instruction for the game
def instruction():
    text1 = font2.render("PRESS D TO RESET TO DEFAULT / R TO EMPTY", 1, (0, 0, 0))
    text2 = font2.render("ENTER VALUES AND PRESS ENTER TO VISUALIZE", 1, (0, 0, 0))
    screen.blit(text1, (20, 520))       
    screen.blit(text2, (20, 540))
 
# Display options when solved
def result():
    text1 = font1.render("FINISHED PRESS R or D", 1, (0, 0, 0))
    screen.blit(text1, (20, 570))   
run = True
flag1 = 0
flag2 = 0
rs = 0
error = 0
# The loop thats keep the window running
while run:
     
    # White color background
    screen.fill((255, 255, 255))
    # Loop through the events stored in event.get()
    for event in pygame.event.get():
        # Quit the game window
        if event.type == pygame.QUIT:
            run = False 
        # Get the mouse position to insert number   
        if event.type == pygame.MOUSEBUTTONDOWN:
            flag1 = 1
            pos = pygame.mouse.get_pos()
            get_cord(pos)
        # Get the number to be inserted if key pressed   
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                x-= 1
                flag1 = 1
            if event.key == pygame.K_RIGHT:
                x+= 1
                flag1 = 1
            if event.key == pygame.K_UP:
                y-= 1
                flag1 = 1
            if event.key == pygame.K_DOWN:
                y+= 1
                flag1 = 1   
            if event.key == pygame.K_1:
                val = 1
            if event.key == pygame.K_2:
                val = 2   
            if event.key == pygame.K_3:
                val = 3
            if event.key == pygame.K_4:
                val = 4
            if event.key == pygame.K_5:
                val = 5
            if event.key == pygame.K_6:
                val = 6
            if event.key == pygame.K_7:
                val = 7
            if event.key == pygame.K_8:
                val = 8
            if event.key == pygame.K_9:
                val = 9 
            if event.key == pygame.K_RETURN:
                flag2 = 1  
            # If R pressed clear the sudoku board
            if event.key == pygame.K_r:
                rs = 0
                error = 0
                flag2 = 0
                grid =[
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0]
                ]
            # If D is pressed reset the board to default
            if event.key == pygame.K_d:
                rs = 0
                error = 0
                flag2 = 0
                grid = deepcopy(grid0)
    if flag2 == 1:
        if solve(grid, 0, 0)== False:
            error = 1
        else:
            rs = 1
        flag2 = 0   
    if val != 0:           
        draw_val(val)
        # print(x)
        # print(y)
        if valid(grid, int(x), int(y), val)== True:
            grid[int(x)][int(y)]= val
            flag1 = 0
        else:
            grid[int(x)][int(y)]= 0
            raise_error2()  
        val = 0   
       
    if error == 1:
        raise_error1() 
    if rs == 1:
        result()       
    draw() 
    if flag1 == 1:
        draw_box()      
    instruction()   
  
    # Update window
    pygame.display.update() 
 
# Quit pygame window   
pygame.quit() 
